In [1]:
#in this notebook we will implement MOCO paper:
#https://arxiv.org/pdf/1911.05722.pdf
#CNN architecture we change, but MOCO loss function is the same, and training is the same
#we use 5 different data gmentations (rotations, blur, color distortion, cropping and resizing) for defining the positive samples

#training is done on 1 GPU, training setting are:
#1. Use the entire training data to learn the representations.
#2. Once the representations are learned, use a linear and logistic layers and retrain with 10-50% of supervised training data.
#3. Experiment with two different sizes for the encoder dictionary.

In [2]:
version = 7
experiment = 'moco_split_norm'
experiment_name = experiment + '_' + str(version)

In [3]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary

from functools import partial
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import numpy as np

In [4]:
from PIL import Image
#tensorboard
from torch.utils.tensorboard import SummaryWriter


In [5]:
import importlib

import models
importlib.reload(models)
from models import  pentaClassifier, binaryClassifier, baseClassifier, convNet


import config
from config import cfg
importlib.reload(config)



<module 'config' from '/home/lisa/bhartendu/adrl/A3/config.py'>

In [6]:
#device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### CIFAR10

In [7]:
from torchvision.datasets import CIFAR10

In [8]:
import argparse
import json
import math
import os
import pandas as pd

In [9]:
#seed
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)



## HYPER - PARAMETERS

In [10]:
results_dir = 'results/'
moco_pretrained_dir = results_dir+'model_'+experiment_name

#### MOCO - HYPER - PARAMETERS

In [11]:
moco_dim = 512
moco_k = 4096
moco_m = 0.99
moco_t = 0.07
moco_loss_symmetric = True

#### MOCO - TRAINING

In [12]:
lr = 0.06
epochs = 200  # 200
batch_size = 512
schedule = [120, 160]
wd = 5e-4
cos = True      # cosine lr schedule
bn_splits = 2

#### Supervised - Hyper - Parameters

In [13]:
fraction_training_data = 0.50

#### Supervised Training

In [14]:
super_lr = 5.0
super_epochs = 10


## data-loader

#### Adapted from pytorch code of Contrastive learning libs: http://github.com/zhirongw/lemniscate.pytorch

In [15]:
class CIFAR10Pair(CIFAR10):
    """CIFAR10 Dataset.
    """
    def __getitem__(self, index):
        img = self.data[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            im_1 = self.transform(img)
            im_2 = self.transform(img)

        return im_1, im_2

# train_transform = transforms.Compose([
#     transforms.RandomResizedCrop(32),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
#     transforms.RandomGrayscale(p=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
#we will aplly 5 different data gmentations (rotations, blur, color distortion, cropping and resizing) for defining the positive samples
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    #random rotation with random angle
    transforms.RandomRotation([-1* torch.rand(1)*360 , torch.rand(1)*360]),
    #random blur
    transforms.RandomApply([transforms.GaussianBlur(3, sigma=[0.1, 2.0])], p=0.5),
    #random color distortion
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])
    

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

# data prepare
train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)

memory_data = CIFAR10(root='data', train=True, transform=test_transform, download=True)
memory_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Split Batch implementation adapted from: 
#### https://timm.fast.ai/tutorial_splitbn

In [16]:
class SplitBatchNorm2d(torch.nn.BatchNorm2d):

    def __init__(self, num_features, eps=1e-5, momentum=0.1, affine=True,
                 track_running_stats=True, num_splits=2):
        super().__init__(num_features, eps, momentum, affine, track_running_stats)
        assert num_splits > 1, 'Should have at least one aux BN layer (num_splits at least 2)'
        self.num_splits = num_splits
        self.aux_bn = nn.ModuleList([
            nn.BatchNorm2d(num_features, eps, momentum, affine, track_running_stats) for _ in range(num_splits - 1)])

    def forward(self, input: torch.Tensor):
        if self.training:  # aux BN only relevant while training
            split_size = input.shape[0] // self.num_splits
            assert input.shape[0] == split_size * self.num_splits, "batch size must be evenly divisible by num_splits"
            split_input = input.split(split_size)
            x = [super().forward(split_input[0])]
            for i, a in enumerate(self.aux_bn):
                x.append(a(split_input[i + 1]))
            return torch.cat(x, dim=0)
        else:
            return super().forward(input)

#### functionality to replace BatchNorm with SplitBatchNorm

In [17]:
#write a function to replace batchnorm with splitbatchnorm
def internal_replace_batch_norm_w_split_norm(model, num_splits = 2):
    #if num_splits is 1, then we will not replace the batchnorm with splitbatchnorm, return the model as it is
    if num_splits == 1:
        return model
    out = False

    for parent_module in model.modules():
        #iterate over all the children of the parent module
        for child_module in parent_module.children():
            #if the child module is batchnorm, then replace it with splitbatchnorm
            if isinstance(child_module, torch.nn.BatchNorm2d):
                #first make copy of all the parameters of the batchnorm
                num_features = child_module.num_features
                eps = child_module.eps
                momentum = child_module.momentum
                affine = child_module.affine
                track_running_stats = child_module.track_running_stats
                #now replace the batchnorm with splitbatchnorm
                parent_module._modules[str(child_module)] = SplitBatchNorm2d(num_features, eps, momentum, affine, track_running_stats, num_splits)
                out = True
                # if out is true break the loop
                if out:
                    break
                
        # break if out is true
        if out:
            break
    return model, out
    

#function to count the number of batchnorm layers in the model
def count_bn_layers(model):
    count = 0
    for parent_module in model.modules():
        for child_module in parent_module.children():
            if isinstance(child_module, torch.nn.BatchNorm2d):
                count += 1
    return count


#write a function to replace splitbatchnorm with batchnorm by using both functions defined above
def batch_norm_w_split_norm(model , num_splits= 2, verbose = False):
    #if num_splits is 1, then we will not replace the batchnorm with splitbatchnorm, return the model as it is
    if num_splits == 1:
        return model
    #first get the number of batchnorm layers in the model
    num_bn_layers = count_bn_layers(model)
    #if the number of batchnorm layers is 0, then return the model as it is
    if num_bn_layers == 0:
        return model
    #if verbose then print the number of batchnorm layers
    if verbose:
        print("Number of batchnorm layers in the model: ", num_bn_layers)

    #if the number of batchnorm layers is not 0, then replace the batchnorm with splitbatchnorm using the function replace_batch_norm_w_split_norm : internal_replace_batch_norm_w_split_norm
    for i in range(num_bn_layers):
        model, out = internal_replace_batch_norm_w_split_norm(model, num_splits)
        #if verbose then print the number of batchnorm layers replaced
        if verbose:
            print("Number of batchnorm layers replaced: ", i+1)
        assert out, "Error in replacing batchnorm with splitbatchnorm"
    return model
    


## Model

In [18]:
#create a model class named ModelBAse that will have the exact same architecture as the convNet class but will flatten the output of the model, to get embedding of the model
class ModelBase(nn.Module):
    def __init__(self, cfg=cfg['model'], device=device):
        super().__init__()
        self.cfg = cfg
        self.device = device
        self.conv= batch_norm_w_split_norm(convNet(cfg, device=device), num_splits=bn_splits)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(moco_dim, moco_dim)
        #replace the batchnorm with splitbatchnorm
        
        
        

        
    def forward(self, x):
        x = self.flat(self.conv(x))
        #pass the output of the model through a linear layer
        x = self.fc(x)

        return x


In [19]:
#buils an object of the ModelBase class
model = ModelBase()


-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [20]:
#print the model
print(model)

ModelBase(
  (conv): convNet(
    (net): ModuleList(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.3, inplace=False)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): ReLU()
      (8): Dropout(p=0.3, inplace=False)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU()
      (13): Dropout(p=0.3, inplace=False)
      (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (BatchNo

In [21]:
# #summary of the model
# #create a dummy input
# dummy_input = torch.randn( 3, 32, 32)


# summary(model, input_size=(8,3, 32, 32), device='cpu')

---------------------------------------------------------------------------
---------------------------------------------------------------------------

## MOCO

#### defininng MOCO Training Routine: Including the queue and the memory bank and key encoder, query encoder




#### the following class was adapted from facebookresearch/moco

In [22]:
class ModelMoCo(nn.Module):
    def __init__(self, dim=moco_dim, K=moco_k, m=moco_m, T=moco_t, bn_splits=bn_splits, symmetric=True):
        super(ModelMoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.symmetric = symmetric

        # create the encoders
        self.encoder_q = ModelBase()
        self.encoder_k = ModelBase()

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1. - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.t()  # transpose
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_single_gpu(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        """
        # random shuffle index
        idx_shuffle = torch.randperm(x.shape[0]).cuda()

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        return x[idx_shuffle], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_single_gpu(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        """
        return x[idx_unshuffle]

    def contrastive_loss(self, im_q, im_k):
        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)  # already normalized

        # compute key features
        with torch.no_grad():  # no gradient to keys
            # shuffle for making use of BN
            im_k_, idx_unshuffle = self._batch_shuffle_single_gpu(im_k)

            k = self.encoder_k(im_k_)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)  # already normalized

            # undo shuffle
            k = self._batch_unshuffle_single_gpu(k, idx_unshuffle)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).cuda()
        
        loss = nn.CrossEntropyLoss().cuda()(logits, labels)

        return loss, q, k

    def forward(self, im1, im2):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            loss
        """

        # update the key encoder
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()

        # compute loss
        if self.symmetric:  # asymmetric loss
            loss_12, q1, k2 = self.contrastive_loss(im1, im2)
            loss_21, q2, k1 = self.contrastive_loss(im2, im1)
            loss = loss_12 + loss_21
            k = torch.cat([k1, k2], dim=0)
        else:  # asymmetric loss
            loss, q, k = self.contrastive_loss(im1, im2)

        self._dequeue_and_enqueue(k)

        return loss



#### Optimizer: SGD

In [23]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)

##### learning rate schedule

In [24]:
# lr scheduler for training
def adjust_learning_rate(optimizer, epoch):
    """Decay the learning rate based on schedule"""
    learning_rate = lr 
    if cos:  # cosine lr schedule
        learning_rate *= 0.5 * (1. + math.cos(math.pi * epoch / epochs))
    else:  # stepwise lr schedule
        for milestone in schedule:
            learning_rate *= 0.1 if epoch >= milestone else 1.
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate

#### Training Loop

In [25]:
# train for one epoch
#create batch writer
batch_writer = SummaryWriter(f'{results_dir}/run/')
batch_count = 0


def train(net, data_loader, train_optimizer, epoch):
    net.train()
    adjust_learning_rate(optimizer, epoch)

    total_loss, total_num, train_bar = 0.0, 0, tqdm(data_loader)
    for im_1, im_2 in train_bar:
        im_1, im_2 = im_1.cuda(non_blocking=True), im_2.cuda(non_blocking=True)

        loss = net(im_1, im_2)
        
        train_optimizer.zero_grad()
        loss.backward()
        train_optimizer.step()

        #write to batch writer
        #make batch count static
        global batch_count
        batch_writer.add_scalar('Loss/train_batch', loss, batch_count)
        batch_count += 1
        ########

        total_num += data_loader.batch_size
        total_loss += loss.item() * data_loader.batch_size
        train_bar.set_description('Train Epoch: [{}/{}], lr: {:.6f}, Loss: {:.4f}'.format(epoch, epochs, optimizer.param_groups[0]['lr'], total_loss / total_num))

    return total_loss / total_num



2022-12-04 10:00:50.563170: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Train

##### instantiate the model

In [26]:
# create model
model = ModelMoCo(
        dim=moco_dim,
        K=moco_k,
        m=moco_m,
        T=moco_t,

        bn_splits=bn_splits,
        symmetric=moco_loss_symmetric,
    ).to(device)
print(model.encoder_q)

-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [27]:
# logging
results = {'train_loss': []}
#if results_dir not exists, then create one
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
#initialize summary writer in run directory in results_dir
writer = SummaryWriter(f'{results_dir}/run/')

##### training loop

In [28]:
# training loop
epoch_start = 1
for epoch in range(epoch_start, epochs + 1):
    
    train_loss = train(model, train_loader, optimizer, epoch)
    results['train_loss'].append(train_loss)
    writer.add_scalar('Loss/train_epoch', train_loss, epoch)
    #if last epoch save model
    if epoch == epochs:
        torch.save(model.state_dict(), moco_pretrained_dir+'.pth')

  0%|          | 0/97 [00:04<?, ?it/s]


RuntimeError: running_mean should contain 128 elements not 32

In [ ]:
# #write a function to load the trained moco model, 
# def load_moco_model(path = moco_pretrained_dir):
#     model = ModelMoCo(
#         dim=moco_dim,
#         K=moco_k,
#         m=moco_m,
#         T=moco_t,

#         bn_splits=bn_splits,
#         symmetric=moco_loss_symmetric,
#     )
#     model.load_state_dict(torch.load(path))
#     return model

In [ ]:
#load saved moco model but return only the encoder_q
def load_moco_model(path = moco_pretrained_dir):
    model = ModelMoCo(
        dim=moco_dim,
        K=moco_k,
        m=moco_m,
        T=moco_t,

        bn_splits=bn_splits,
        symmetric=moco_loss_symmetric,
    )
    model.load_state_dict(torch.load(path+'.pth'))
    return model.encoder_q


# Supervised Training

# Using the Learned Representations

In [ ]:
#we will test using the standard Linear classification Protocol
#the Linear classification Protocol is : as unsupervised representations are learned, use a linear and logistic layers and retrain with 10-50% of supervised training data

#we have 2 test tasks 
#TASK 1: Binary Classifier
#TASk 2: 5-class classification

In [ ]:
#we define a classifier class, that takes in number of classes
#it will first make a copy of the ModelMoCo class, load the pretrained weights, make the weights non-trainable
#then we will add a linear layer and a logistic layer to the model to match the number of classes
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.moco = load_moco_model()
        #freeze the weights
        for param in self.moco.parameters():
            param.requires_grad = False
        #add a linear layer and a softmax layer
        self.linear = nn.Linear(moco_dim, num_classes)
        #add logistic layer
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.moco(x)
        x = self.linear(x)
        x = self.softmax(x)
        return x


In [ ]:
#print summary of the model
model = Classifier(2)
summary(model.to(device), (3, 32, 32))

-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

In [ ]:
print(model)

Classifier(
  (moco): ModelBase(
    (conv): convNet(
      (net): ModuleList(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.3, inplace=False)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): ReLU()
        (8): Dropout(p=0.3, inplace=False)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (12): ReLU()
        (13): Dropout(p=0.3, inplace=False)
        (14): MaxPool2d(kernel_size=2, stride=2

#### dataloader for supervised training

In [ ]:
#define train and test loaders
#get train loader only fraction_trainig_data
#we load cifar10 dataset from torchvision
train_data_supervised = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=test_transform)
#take only fraction of training data : fraction_training_data 
train_data_supervised = torch.utils.data.Subset(train_data_supervised, indices = np.random.choice(len(train_data_supervised), int(fraction_training_data*len(train_data_supervised)), replace=False))

#train dataloader
train_loader_supervised = torch.utils.data.DataLoader(train_data_supervised, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)



Files already downloaded and verified


#### Supervised Training Routine

In [ ]:
#we will have a supervised training loop on a model given by the classifier class
def supervised_training(model,label_fun, model_save_path, epochs= super_epochs, lr):

    #loss function
    criterion = torch.nn.CrossEntropyLoss()
    #optimizer, we will only train the linear and logistic layers, thus we will only pass the parameters of the linear and logistic layers, i.e only the trainable parameters
    #pass only the parameters of last linear layer and softmax layer
    optimizer = torch.optim.SGD(model.linear.parameters(), lr=super_lr)
    #CosineAnnealingLR scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)



    model.train()
    #send model to device
    model.to(device)
    train_bar = tqdm(train_loader)
    #loop for number of epochs
    batch_counter = 0
    for epoch in range(epochs):
        epoch_loss = 0
        total_correct = 0
        total_items = 0
        #loop for number of batches
        for batch_idx, (data, target) in enumerate(train_loader_supervised):
            #send data and target to device
            data, target = data.to(device), target
            #get binary target
            target = label_fun(target)
            #send to device
            target = target.to(device)
            #zero the gradients
            optimizer.zero_grad()
            #forward pass
            output = model(data)
            #calculate loss
            loss = criterion(output, target)
            #backward pass
            loss.backward()
            #update weights
            optimizer.step()
            #update epoch loss
            epoch_loss += loss.item()
            #get classification accuracy
            _, predicted = torch.max(output.data, 1)
            total = target.size(0)
            correct = (predicted == target).sum().item()
            total_correct += correct
            total_items += total
            #update progress bar
            # train_bar.set_description(desc= f'epoch {epoch} loss={loss.item()} batch_id={batch_idx} Accuracy={100 * correct / total}')
            #print
            if batch_idx % 25 == 0:
                print('Epoch : %d/%d, Batch : %d/%d, Loss : %.4f, Accuracy : %.4f'
                    %(epoch+1, epochs, batch_idx+1, len(train_loader_supervised), loss.item(), 100 * correct / total))

            #write to tensorboard
            writer.add_scalar('Loss/batch_train_supervised_binary_classifier', loss.item(), batch_counter)
            writer.add_scalar('Accuracy/batch_train_supervised_binary_classifier', 100 * correct / total, batch_counter)
            batch_counter += 1

        #update scheduler
        scheduler.step()
        #print epoch loss
        print("epoch loss",epoch_loss/len(train_loader))
        #print epoch accuracy
        print("epoch accuracy",total_correct/total_items)
        #write to tensorboard
        writer.add_scalar('Loss/epoch_train_supervised_binary_classifier', epoch_loss/len(train_loader), epoch)
        writer.add_scalar('Accuracy/epoch_train_supervised_binary_classifier', total_correct/total_items, epoch)
        #if last epoch , then save the model
        if epoch == epochs-1:
            torch.save(model.state_dict(), model_save_path+'.pt')
    return model



    


## TASK 0: CIFAR10 10-class classification

In [ ]:
#define 10 class label
def ten_class_label(ip):
    return ip

In [ ]:
#we will call the supervised training function for 10 class classification
#we will build the parameters for the function
#model
model = Classifier(10)
model_save_path = moco_pretrained_dir + '_supervised_ten_class_classifier'
#call the function
model = supervised_training(model, ten_class_label, model_save_path, epochs= super_epochs)


## TASK 1: Binary Classifier

In [ ]:
#we will define binary label function that will map the input label: 0 to 9 to either 0 or 1
#y = torch.tensor([1 if x in [2,3,4,5,6,7] else 0 for x in y])
def binary_label(y):
    y = torch.tensor([1 if x in [2,3,4,5,6,7] else 0 for x in y])
    return y

In [ ]:
#we will call the supervised training function for binary classification
#we will build the parameters for the function
#model
model = Classifier(2)
model_save_path = moco_pretrained_dir + '_supervised_binary_classifier'
#call the function
model = supervised_training(model, binary_label, model_save_path, epochs= super_epochs)

-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

Epoch : 1/10, Batch : 1/49, Loss : 0.7013, Accuracy : 50.7812
Epoch : 1/10, Batch : 26/49, Loss : 0.6825, Accuracy : 58.3984
epoch loss 0.345010670190005
epoch accuracy 0.57452
Epoch : 2/10, Batch : 1/49, Loss : 0.6617, Accuracy : 64.8438
Epoch : 2/10, Batch : 26/49, Loss : 0.6462, Accuracy : 68.1641
epoch loss 0.3322585550780149
epoch accuracy 0.63704
Epoch : 3/10, Batch : 1/49, Loss : 0.6443, Accuracy : 64.2578
Epoch : 3/10, Batch : 26/49, Loss : 0.6408, Accuracy : 64.6484
epoch loss 0.32388947735127716
epoch accuracy 0.66048
Epoch : 4/10, Batch : 1/49, Loss : 0.6489, Accuracy : 61.5234
Epoch : 4/10, Batch : 26/49, Loss : 0.6198, Accuracy : 70.3125
epoch loss 0.317789346286931
epoch accuracy 0.67896
Epoch : 5/10, Batch : 1/49, Loss : 0.6287, Accuracy : 67.1875
Epoch : 5/10, Batch : 26/49, Loss : 0.6157, Accuracy : 69.9219
epoch loss 0.3127829358749783
epoch accuracy 0.69768
Epoch : 6/10, Batch : 1/49, Loss : 0.6142, Accuracy : 69.5312
Epoch : 6/10, Batch : 26/49, Loss : 0.6134, Accur

  0%|          | 0/97 [00:17<?, ?it/s]

epoch loss 0.30469172271256595
epoch accuracy 0.72648


## TASK 2: Multi-class Classifier

In [ ]:
def five_class_label(ip):
    l = []
    for x in ip:
        if x in [6,2]:
            l.append(0)
        if x in [3,5]:
            l.append(1)
        if x in [4,7]:
            l.append(2)
        if x in [1,9]:
            l.append(3)
        if x in [0,8]:
            l.append(4)
        
    return torch.tensor(l)


In [ ]:
#we will call the supervised training function for multi-classification
#we will build the parameters for the function
#model
model = Classifier(5)
model_save_path = moco_pretrained_dir + '_supervised_five_class_classifier'
#call the function
model = supervised_training(model, five_class_label, model_save_path, epochs= super_epochs)



-----------------------------------------------------------
ConvNet
ModuleList(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): Dropout(p=0.3, inplace=False)
  (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
---------------------------------------------

Epoch : 1/10, Batch : 1/49, Loss : 1.6116, Accuracy : 20.8984
Epoch : 1/10, Batch : 26/49, Loss : 1.6121, Accuracy : 20.3125
epoch loss 0.8131457948193108
epoch accuracy 0.21064
Epoch : 2/10, Batch : 1/49, Loss : 1.6084, Accuracy : 21.6797
Epoch : 2/10, Batch : 26/49, Loss : 1.6124, Accuracy : 20.3125
epoch loss 0.8115099385841605
epoch accuracy 0.22304
Epoch : 3/10, Batch : 1/49, Loss : 1.6057, Accuracy : 22.2656
Epoch : 3/10, Batch : 26/49, Loss : 1.6039, Accuracy : 22.0703
epoch loss 0.8101080139887702
epoch accuracy 0.23328
Epoch : 4/10, Batch : 1/49, Loss : 1.6001, Accuracy : 25.3906
Epoch : 4/10, Batch : 26/49, Loss : 1.6045, Accuracy : 22.2656
epoch loss 0.8087335775807961
epoch accuracy 0.2402
Epoch : 5/10, Batch : 1/49, Loss : 1.6002, Accuracy : 23.4375
Epoch : 5/10, Batch : 26/49, Loss : 1.6017, Accuracy : 25.0000
epoch loss 0.8075221162481406
epoch accuracy 0.25032
Epoch : 6/10, Batch : 1/49, Loss : 1.6014, Accuracy : 22.2656
Epoch : 6/10, Batch : 26/49, Loss : 1.6010, Accur

  0%|          | 0/97 [00:17<?, ?it/s]

epoch loss 0.8052204318882263
epoch accuracy 0.26144


In [ ]:
#good learning rate for 10 class is :1.0